In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import time
import os
import copy
import visdom
import pandas as pd
from math import ceil
from PIL import Image
from sklearn.cluster import KMeans
import shutil
from sklearn.decomposition import PCA


plt.ion()   # interactive mode

In /home/ghayat/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ghayat/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ghayat/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/ghayat/.local/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/ghayat/.local/lib/python3.6/site-packages/matplotlib/mpl-data/s

In [2]:
import sys  
sys.path.insert(0, '../Resnet/')
from model import *
from my_ImageFolder import *
from fairness_metrics import *

In [3]:
path_bask_r_f = '../Datasets/basket_volley/basket/basket_f_r/'
path_bask_y_f = '../Datasets/basket_volley/basket/basket_f_y/'
path_bask_r_m = '../Datasets/basket_volley/basket/basket_m_r/'
path_bask_y_m = '../Datasets/basket_volley/basket/basket_m_y/'

bask_r_f = os.listdir(path_bask_r_f)
bask_y_f = os.listdir(path_bask_y_f)
bask_r_m = os.listdir(path_bask_r_m)
bask_y_m = os.listdir(path_bask_y_m)

path_voll_r_f = '../Datasets/basket_volley/volley/volley_f_r/'
path_voll_y_f = '../Datasets/basket_volley/volley/volley_f_y/'
path_voll_r_m = '../Datasets/basket_volley/volley/volley_m_r/'
path_voll_y_m = '../Datasets/basket_volley/volley/volley_m_y/'

voll_r_f = os.listdir(path_voll_r_f)
voll_y_f = os.listdir(path_voll_y_f)
voll_r_m = os.listdir(path_voll_r_m)
voll_y_m = os.listdir(path_voll_y_m)

In [4]:
class0_min, class1_min = bask_y_m + bask_y_f,voll_r_m + voll_r_f
protected_groups = set(class0_min + class1_min)

In [5]:
W_PROTECTED, BIAS, VAL_MODE, START_EPOCH, NUM_EPOCH, SHOW_PROGRESS, ID, DATASET, NUM_TRIALS = 1, 0.8, False, 0, 10, False, 0, "basket_volley", 1

In [6]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        # transforms.RandomResizedCrop(224),
        # transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '../Datasets/basket_volley/train_test_split'
image_datasets = {
    x: my_ImageFolder(os.path.join(data_dir, f"train_0.8" if x == "train" else x), data_transforms[x],
                      protected_groups, W_PROTECTED)
    for x in ['train', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                              shuffle=True, num_workers=4)
               for x in ['train', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Training Conv net

In [10]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 12, 5)
        self.fc1 = nn.Linear(33708, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, len(class_names))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [11]:
net = Net()
net.fc2.register_forward_hook(get_activation('fc2'))

criterion = nn.CrossEntropyLoss() # weighted_cross_entropy_loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [12]:
model_conv = train_model(net, criterion, optimizer, scheduler, dataloaders, dataset_sizes, device,
                             start_epoch=START_EPOCH,
                             num_epochs=NUM_EPOCH,
                             val_mode=VAL_MODE, show_progress=SHOW_PROGRESS)

Epoch 0/9
----------
train Loss: 0.5731 Acc: 0.7176
Epoch 1/9
----------
train Loss: 0.4885 Acc: 0.7908
Epoch 2/9
----------
train Loss: 0.4165 Acc: 0.8139
Epoch 3/9
----------
train Loss: 0.3525 Acc: 0.8472
Epoch 4/9
----------
train Loss: 0.2766 Acc: 0.8922
Epoch 5/9
----------
train Loss: 0.2175 Acc: 0.9114
Epoch 6/9
----------
train Loss: 0.1384 Acc: 0.9499
Epoch 7/9
----------
train Loss: 0.0613 Acc: 0.9833
Epoch 8/9
----------
train Loss: 0.0328 Acc: 0.9936
Epoch 9/9
----------
train Loss: 0.0220 Acc: 1.0000
Training complete in 20m 19s
Best val Acc: 0.000000


In [13]:
print(f"Acc. on Training set: {float(accuracy(net, device, dataloaders['train']))}")
print(f"Acc. on Test set: {float(accuracy(net, device, dataloaders['test']))}")
print(f"Fairness measures")
demographic_parity(net, device, image_datasets["test"], [class0_min, class1_min]).to_numpy()

Acc. on Training set: 1.0
Acc. on Test set: 0.6136919315403423
Fairness measures


array([[0.94642857, 0.49494949],
       [0.13592233, 0.86315789]])

#### Training Complete Resnet

In [8]:
model_conv = torchvision.models.resnet18(pretrained=True)
for param in model_conv.parameters():
    param.requires_grad = False

In [9]:
model_conv = torchvision.models.resnet18(pretrained=True)

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, len(class_names))

model_conv = model_conv.to(device)

criterion = weighted_cross_entropy_loss  # nn.CrossEntropyLoss()

# Observe that only parameters of final layer are being optimized as
# opposed to before.
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

In [10]:
model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, dataloaders, dataset_sizes, device,
                             start_epoch=START_EPOCH,
                             num_epochs=NUM_EPOCH,
                             val_mode=VAL_MODE, show_progress=SHOW_PROGRESS)

Epoch 0/1
----------
train Loss: 0.2367 Acc: 0.9692
Epoch 1/1
----------
train Loss: 3.7493 Acc: 0.8742
Training complete in 3m 14s
Best val Acc: 0.000000


In [11]:
print(f"Acc. on Training set: {float(accuracy(model_conv, device, dataloaders['train']))}")
print(f"Acc. on Test set: {float(accuracy(model_conv, device, dataloaders['test']))}")
print(f"Fairness measures")
demographic_parity(model_conv, device, image_datasets["test"], [class0_min, class1_min]).to_numpy()

Acc. on Training set: 0.4582798459563543
Acc. on Test set: 0.4841075794621027
Fairness measures


array([[0., 0.],
       [1., 1.]])

In [90]:
model_repr = nn.Sequential(*list(model_conv.children())[:-1])

In [108]:
X_transform = np.array([[]]).reshape(0, model_conv.fc.in_features)
for (inputs, label), weights in dataloaders["train"]:
    output = model_repr(inputs).detach().numpy().reshape((inputs.shape[0], -1))
    X_transform = np.concatenate([X_transform, output])

#### Taking pretrained representation Resnet

In [55]:
X_transform = np.array([[]]).reshape(0,model_conv.fc.out_features)
for (inputs, label), weights in dataloaders["train"]:
    output = model_conv(inputs)
    X_transform = np.concatenate([X_transform, output])
indexes = list(range(len(image_datasets["train"])))

#### Extracting represenation

In [22]:
X_transform, labels, indexes = np.array([[]]).reshape(0,net.fc2.out_features), np.array([]).astype(int), np.array([]).astype(int)
for i, ((inputs, _), index) in enumerate(dataloaders["train"]):
    output = net(inputs)
    output = activation['fc2']
    X_transform = np.concatenate([X_transform, output])
    indexes = np.concatenate([indexes, index.numpy().reshape(-1)])

#### PCA reduction with pretrained repr

In [23]:
X_reducted = PCA(n_components=10).fit_transform(X_transform)
kmeans = KMeans(n_clusters=4, random_state=0).fit_predict(X_reducted)
path = os.path.join(data_dir, "train_0.8")

In [29]:
cluster_paths = view_clusters(path, kmeans, indexes)

#### pretrained statistics 

In [80]:
statistics(path, cluster_paths)

--------------Cluster 0--------- 
 n. samples: 208
 n. of bask: 116 (55.8%)
 n. of volley: 92 (44.2%)
 n. of red: 106 (51.0%)
 n. of yellow: 102 (49.0%)
 n. of males: 106 (51.0%)
 n. of females: 102 (49.0%)
--------------Cluster 1--------- 
 n. samples: 223
 n. of bask: 130 (58.3%)
 n. of volley: 93 (41.7%)
 n. of red: 118 (52.9%)
 n. of yellow: 105 (47.1%)
 n. of males: 102 (45.7%)
 n. of females: 121 (54.3%)
--------------Cluster 2--------- 
 n. samples: 185
 n. of bask: 103 (55.7%)
 n. of volley: 82 (44.3%)
 n. of red: 101 (54.6%)
 n. of yellow: 84 (45.4%)
 n. of males: 91 (49.2%)
 n. of females: 94 (50.8%)
--------------Cluster 3--------- 
 n. samples: 163
 n. of bask: 73 (44.8%)
 n. of volley: 90 (55.2%)
 n. of red: 84 (51.5%)
 n. of yellow: 79 (48.5%)
 n. of males: 68 (41.7%)
 n. of females: 95 (58.3%)


#### Conv statistics

In [31]:
statistics(path, cluster_paths)

--------------Cluster 0--------- 
 n. samples: 172
 n. of bask: 142 (82.6%)
 n. of volley: 30 (17.4%)
 n. of red: 128 (74.4%)
 n. of yellow: 44 (25.6%)
 n. of males: 81 (47.1%)
 n. of females: 91 (52.9%)
--------------Cluster 1--------- 
 n. samples: 280
 n. of bask: 280 (100.0%)
 n. of volley: 0 (0.0%)
 n. of red: 228 (81.4%)
 n. of yellow: 52 (18.6%)
 n. of males: 130 (46.4%)
 n. of females: 150 (53.6%)
--------------Cluster 2--------- 
 n. samples: 238
 n. of bask: 0 (0.0%)
 n. of volley: 238 (100.0%)
 n. of red: 37 (15.5%)
 n. of yellow: 201 (84.5%)
 n. of males: 111 (46.6%)
 n. of females: 127 (53.4%)
--------------Cluster 3--------- 
 n. samples: 89
 n. of bask: 0 (0.0%)
 n. of volley: 89 (100.0%)
 n. of red: 16 (18.0%)
 n. of yellow: 73 (82.0%)
 n. of males: 45 (50.6%)
 n. of females: 44 (49.4%)


#### Helper functions

In [28]:
def view_clusters(path, kmeans, indexes):
    K = len(set(kmeans))
    bask, voll = os.listdir(os.path.join(path, "basket")), os.listdir(os.path.join(path, "volley"))
    
    paths = []
    for k in range(K):
        paths.append(f"clustering_{K}/cluster_{k}")
        os.makedirs(paths[-1], exist_ok=True)
        
    for i in range(len(kmeans)):
        src = os.path.join(path, "basket/" if indexes[i] < len(bask) else "volley/") + (bask[indexes[i]] if indexes[i] < len(bask) else voll[indexes[i] - len(bask)])
        dst = f"clustering_{K}/cluster_{kmeans[i]}/" + (bask[indexes[i]] if indexes[i] < len(bask) else voll[indexes[i] - len(bask)])
        shutil.copy(src, dst)
    
    return paths

def statistics(path, clusters):
    K = len(set(clusters))
    dr, nur = os.listdir(os.path.join(path, "basket")), os.listdir(os.path.join(path, "volley"))
    
    for k in range(K):
        n_bask, n_voll, n_r, n_y, n_m, n_f = 0, 0, 0, 0, 0, 0
        cluster = os.listdir(clusters[k])
        for img in cluster:
            if img in bask_r_f:
                n_bask += 1
                n_f += 1
                n_r += 1
                
            if img in bask_r_m:
                n_bask += 1
                n_m += 1
                n_r += 1
                
            if img in bask_y_f:
                n_bask += 1
                n_f += 1
                n_y += 1
            
            if img in bask_y_m:
                n_bask += 1
                n_m += 1
                n_y += 1
            
            if img in voll_r_f:
                n_voll += 1
                n_f += 1
                n_r += 1
            
            if img in voll_r_m:
                n_voll += 1
                n_m += 1
                n_r += 1
                
            if img in voll_y_f:
                n_voll += 1
                n_f += 1
                n_y += 1
            
            if img in voll_y_m:
                n_voll += 1
                n_m += 1
                n_y += 1
                
        
        print(f"--------------Cluster {k}--------- \n n. samples: {len(cluster)}\n n. of bask: {n_bask} ({n_bask/len(cluster)*100:.1f}%)\n n. of volley: {n_voll} ({n_voll/len(cluster)*100:.1f}%)\n n. of red: {n_r} ({n_r/len(cluster)*100:.1f}%)\n n. of yellow: {n_y} ({n_y/len(cluster)*100:.1f}%)\n n. of males: {n_m} ({n_m/len(cluster)*100:.1f}%)\n n. of females: {n_f} ({n_f/len(cluster)*100:.1f}%)")